# Trabalho de Otimizacao: Minimizacao de Energia de Trajetoria de um Robo
Aluno: Guilherme Fernandes Monteiro (22403229)

Notebook revisado com correcoes matematicas sugeridas na avaliacao.

In [ ]:
# Configura o ambiente garantindo que dependencias basicas estejam instaladas
import sys
import subprocess
from pathlib import Path

# Funcao utilitaria que garante que todas as instalacoes usem o mesmo interpretador do notebook
def run_pip(args):
    cmd = [sys.executable, '-m', 'pip'] + args
    print('Executando:', ' '.join(cmd))
    subprocess.check_call(cmd)

# Garante que o pip esteja disponivel mesmo em distribuicoes minimalistas de Python
try:
    import pip  # noqa: F401
except ImportError:
    import ensurepip
    print('Pip nao encontrado; executando ensurepip...')
    ensurepip.bootstrap()

# Atualiza ferramentas basicas de empacotamento para evitar conflitos
try:
    run_pip(['install', '--upgrade', 'pip', 'setuptools', 'wheel'])
except subprocess.CalledProcessError as err:
    print('Aviso: nao foi possivel atualizar pip/setuptools/wheel automaticamente:', err)

req_path = Path('requirements.txt')
if req_path.exists():
    try:
        run_pip(['install', '-r', str(req_path)])
    except subprocess.CalledProcessError as err:
        print('Falha ao instalar requisitos via arquivo. Tentando instalar pacote a pacote...')
        for raw_pkg in req_path.read_text(encoding='utf-8').splitlines():
            pkg = raw_pkg.strip()
            if not pkg or pkg.startswith('#'):
                continue
            try:
                run_pip(['install', pkg])
            except subprocess.CalledProcessError as inner_err:
                print(f'Aviso: nao foi possivel instalar {pkg}: {inner_err}')
else:
    run_pip(['install', 'numpy', 'scipy', 'matplotlib', 'pyomo'])


## Objetivo
Modelar a trajetoria plana de um robo que se move do ponto inicial (0, 0) ao ponto final (10, 10) minimizando a energia consumida ao longo do caminho. A energia total e aproximada pela integral da soma entre o quadrado da velocidade e o termo sinusoidal `sin(t)`, discretizada pelo metodo dos trapezios.

### Passo 1 - Configuracao do experimento
Definimos os parametros principais da simulacao (numero de intervalos, tamanho do passo de tempo e malha temporal) e importamos as bibliotecas utilizadas.

In [ ]:
# Importa bibliotecas usadas na modelagem matematica e na visualizacao
from pyomo.environ import (
    ConcreteModel,
    RangeSet,
    Var,
    Reals,
    Objective,
    minimize,
    SolverFactory,
    value,
)
import numpy as np
import matplotlib.pyplot as plt
from time import time

# Parametros principais da discretizacao temporal: quantidade de intervalos e passo constante
N = 5  # numero de subintervalos
dt = 1.0  # tamanho do passo de tempo
t_vals = list(range(N + 1))  # lista com todos os instantes discretos avaliados

# Feedback rapido para confirmar a configuracao temporal utilizada nas demais analises
print(f"Numero de intervalos (N): {N}")
print(f"Tamanho do passo (dt): {dt}")
print(f"Pontos de tempo avaliados: {t_vals}")


### Passo 2 - Definicao do modelo Pyomo
Criamos o modelo, declaramos o conjunto de indices para o tempo e as variaveis de decisao correspondentes as coordenadas `x(t)` e `y(t)`. As condicoes de contorno garantem que o robo saia de `(0, 0)` e chegue a `(10, 10)`.

In [ ]:
# Constroi o modelo Pyomo e declara variaveis de decisao para cada instante discreto
model = ConcreteModel()
model.t = RangeSet(0, N)  # conjunto de indices representando os instantes discretos
model.x = Var(model.t, domain=Reals)
model.y = Var(model.t, domain=Reals)

# condicoes de contorno: impoem as posicoes inicial e final do robo
model.x[0].fix(0.0)
model.y[0].fix(0.0)
model.x[N].fix(10.0)
model.y[N].fix(10.0)

print('Modelo criado com variaveis de posicao para cada instante discreto.')


### Passo 3 - Funcao objetivo via metodo dos trapezios
A energia e calculada somando, em cada intervalo `[t_i, t_{i+1}]`, a media dos termos `v^2 + sin(t)` nas extremidades multiplicada por `dt`. A funcao `v_quadrado` computa a velocidade ao quadrado usando diferencas finitas.

In [ ]:
# Funcoes auxiliares para calcular velocidade e energia via metodo dos trapezios
def v_quadrado(model, i):
    '''Calcula a velocidade ao quadrado no intervalo i.'''
    dx = (model.x[i + 1] - model.x[i]) / dt
    dy = (model.y[i + 1] - model.y[i]) / dt
    return dx**2 + dy**2

def regra_energia(model):
    total = 0.0  # acumula a energia total integrada ao longo da trajetoria
    for i in range(N):  # percorre cada subintervalo temporal
        vi_sq = v_quadrado(model, i)
        # no ultimo intervalo reutilizamos o mesmo valor para evitar acessar indice inexistente
        vi1_sq = v_quadrado(model, i + 1) if i < N - 1 else vi_sq
        # aplica a regra do trapezio: media dos valores nas extremidades multiplicada pelo passo dt
        sin_term = 0.5 * ((vi_sq + np.sin(t_vals[i])) + (vi1_sq + np.sin(t_vals[i + 1])))
        total += sin_term * dt
    return total

model.energy = Objective(rule=regra_energia, sense=minimize)

print('Funcao objetivo registrada com integracao trapezoidal da energia.')


### Passo 4 - Resolucao numerica com IPOPT
Utilizamos o solver `ipopt` (instalado via Miniconda) para resolver o problema de programacao nao linear.

In [ ]:
# Configura e executa o solver IPOPT, registrando tempo e diagnosticos
from pathlib import Path
from pyomo.opt import TerminationCondition

# Localiza o executavel instalado via Miniconda (ajuste se necessario)
ipopt_exec = Path.home() / 'miniconda3' / 'Library' / 'bin' / 'ipopt.exe'
if not ipopt_exec.exists():
    raise FileNotFoundError(f"Executavel IPOPT nao encontrado em {ipopt_exec}")

# Configura o solver apontando explicitamente para o executavel desejado
solver = SolverFactory('ipopt', executable=str(ipopt_exec))
if not solver.available(exception_flag=False):
    raise RuntimeError('Solver IPOPT nao disponivel para o Pyomo.')

# Executa a otimizacao e cronometra o tempo de processamento
inicio = time()
resultado = solver.solve(model)
tempo_exec = time() - inicio

# Registra status e condicao de parada devolvidos pelo IPOPT
status = resultado.solver.status
condicao = resultado.solver.termination_condition

print(f"Status do solver: {status}")
print(f"Condicao de parada: {condicao}")
print(f"Tempo de execucao: {tempo_exec:.6f} segundos")

if condicao != TerminationCondition.optimal:
    print("Aviso: condicao de parada diferente de 'optimal'. Revise logs do solver.")


### Passo 5 - Analise numerica e visual
Com o modelo resolvido, avaliamos o valor da energia minimizada, listamos as coordenadas otimas e plotamos a trajetoria resultante.

In [ ]:
# Valor da funcao objetivo otima
energia_total = value(model.energy)
print(f"Energia total minimizada: {energia_total:.6f}")

# Posicoes otimas calculadas para cada instante
print('
Posicoes otimas:')
for i in model.t:
    print(f"t={i}: x={value(model.x[i]):.3f}, y={value(model.y[i]):.3f}")

# Vetores auxiliares para gerar o grafico da trajetoria
x_vals = [value(model.x[t]) for t in model.t]
y_vals = [value(model.y[t]) for t in model.t]

# Plota a trajetoria do robo no plano cartesiano
plt.figure(figsize=(6, 6))
plt.plot(x_vals, y_vals, marker='o', linestyle='-')
plt.title('Trajetoria Otima do Robo')
plt.xlabel('x')
plt.ylabel('y')
plt.grid(True)
plt.axis('equal')
plt.tight_layout()
plt.show()

print('
Observacao: com N=5 o caminho resultado e essencialmente uma linha reta.')


## Experimentos extrasNesta secao exploramos variantes do problema original para compreender como os parametros afetam a trajetoria otimizada.

### Sensibilidade ao numero de intervalosAumentamos o valor de `N` para refinar a discretizacao temporal, avaliando a influencia do termo `sin(t)` sobre a trajetoria.

In [ ]:
# Analisa a trajetoria para diferentes numeros de subintervalos
from collections import OrderedDict

# Reconstrui o modelo com um numero arbitrario de subintervalos mantendo o horizonte total
def resolver_para_n(n_subintervalos):
    modelo = ConcreteModel()
    modelo.t = RangeSet(0, n_subintervalos)
    modelo.x = Var(modelo.t, domain=Reals)
    modelo.y = Var(modelo.t, domain=Reals)
    modelo.x[0].fix(0.0)
    modelo.y[0].fix(0.0)
    modelo.x[n_subintervalos].fix(10.0)
    modelo.y[n_subintervalos].fix(10.0)

    passo = (dt * N) / n_subintervalos  # ajusta o passo para preservar o tempo final de 5s
    tempos = [i * passo for i in range(n_subintervalos + 1)]

    def vel2(m, i):
        dx = (m.x[i + 1] - m.x[i]) / passo
        dy = (m.y[i + 1] - m.y[i]) / passo
        return dx**2 + dy**2

    def energia(m):
        total = 0.0
        for i in range(n_subintervalos):
            v_i = vel2(m, i)
            v_next = vel2(m, i + 1) if i < n_subintervalos - 1 else v_i
            sin_term = 0.5 * ((v_i + np.sin(tempos[i])) + (v_next + np.sin(tempos[i + 1])))
            total += sin_term * passo
        return total

    modelo.energy = Objective(rule=energia, sense=minimize)
    resultado = solver.solve(modelo)
    x_vals_local = [value(modelo.x[t]) for t in modelo.t]
    y_vals_local = [value(modelo.y[t]) for t in modelo.t]
    return {
        'condicao': resultado.solver.termination_condition,
        'energia': value(modelo.energy),
        'x': x_vals_local,
        'y': y_vals_local,
        'tempos': tempos,
    }

# Calcula a energia para diferentes discretizacoes
resultados_n = OrderedDict()
for n_test in (5, 10, 20, 40):
    resultados_n[n_test] = resolver_para_n(n_test)

# Relatorio resumido dos resultados obtidos
for n_test, info in resultados_n.items():
    print(f"N={n_test:2d} :: Energia {info['energia']:.4f} :: Condicao {info['condicao']}")


In [ ]:
# Visualiza as trajetorias para cada discretizacao
plt.figure(figsize=(6, 6))
# Sobrepoe cada trajetoria para observar diferencas geometricas
for n_test, info in resultados_n.items():
    plt.plot(info['x'], info['y'], marker='o', linestyle='-', label=f"N={n_test}")

plt.title('Sensibilidade da Trajetoria em relacao a N')
plt.xlabel('x')
plt.ylabel('y')
plt.axis('equal')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


### Restricao intermediaria personalizadaForcamos a passagem pelo ponto intermediario `(5, 6)` no tempo central, simulando uma restricao de obstaculo ou checkpoint.

In [ ]:
# Adiciona uma restricao de passagem por ponto intermediario
modelo_restrito = ConcreteModel()
modelo_restrito.t = RangeSet(0, N)
modelo_restrito.x = Var(modelo_restrito.t, domain=Reals)
modelo_restrito.y = Var(modelo_restrito.t, domain=Reals)

# Condicoes de contorno herdadas do problema original
modelo_restrito.x[0].fix(0.0)
modelo_restrito.y[0].fix(0.0)
modelo_restrito.x[N].fix(10.0)
modelo_restrito.y[N].fix(10.0)

# Impoe um ponto intermediario obrigatorio, simulando requisito operacional
mid_index = N // 2
modelo_restrito.x[mid_index].fix(5.0)
modelo_restrito.y[mid_index].fix(6.0)

def vel2_restrito(m, i):
    dx = (m.x[i + 1] - m.x[i]) / dt
    dy = (m.y[i + 1] - m.y[i]) / dt
    return dx**2 + dy**2

def energia_restrita(m):
    total = 0.0
    for i in range(N):
        v_i = vel2_restrito(m, i)
        v_next = vel2_restrito(m, i + 1) if i < N - 1 else v_i
        sin_term = 0.5 * ((v_i + np.sin(t_vals[i])) + (v_next + np.sin(t_vals[i + 1])))
        total += sin_term * dt
    return total

modelo_restrito.energy = Objective(rule=energia_restrita, sense=minimize)
resultado_restrito = solver.solve(modelo_restrito)

# Extrai a trajetoria resultante para comparacao
x_restrito = [value(modelo_restrito.x[t]) for t in modelo_restrito.t]
y_restrito = [value(modelo_restrito.y[t]) for t in modelo_restrito.t]
energia_restrita = value(modelo_restrito.energy)

print(f"Energia com ponto intermediario: {energia_restrita:.6f}")
print(f"Condicao do solver: {resultado_restrito.solver.termination_condition}")


In [ ]:
# Compara a trajetoria original com a trajetoria restrita
plt.figure(figsize=(6, 6))
plt.plot(x_vals, y_vals, marker='o', linestyle='-', label='Trajetoria original')
plt.plot(x_restrito, y_restrito, marker='s', linestyle='--', label='Com restricao')
plt.scatter([5.0], [6.0], color='red', zorder=5, label='Ponto imposto')
plt.title('Impacto da Restricao Intermediaria')
plt.xlabel('x')
plt.ylabel('y')
plt.axis('equal')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


### Penalizacao alternativa na funcao objetivoIncluimos um termo quadratico que aproxima o caminho do alvo ao longo do horizonte, privilegiando trajetorias que avancem continuamente para `(10, 10)`.

In [ ]:
# Ajusta a funcao objetivo com termo de penalizacao de distancia
modelo_mod = ConcreteModel()
modelo_mod.t = RangeSet(0, N)
modelo_mod.x = Var(modelo_mod.t, domain=Reals)
modelo_mod.y = Var(modelo_mod.t, domain=Reals)

modelo_mod.x[0].fix(0.0)
modelo_mod.y[0].fix(0.0)
modelo_mod.x[N].fix(10.0)
modelo_mod.y[N].fix(10.0)

def vel2_mod(m, i):
    dx = (m.x[i + 1] - m.x[i]) / dt
    dy = (m.y[i + 1] - m.y[i]) / dt
    return dx**2 + dy**2

def energia_mod(m):
    total = 0.0
    for i in range(N):
        v_i = vel2_mod(m, i)
        v_next = vel2_mod(m, i + 1) if i < N - 1 else v_i
        sin_term = 0.5 * ((v_i + np.sin(t_vals[i])) + (v_next + np.sin(t_vals[i + 1])))
        penal = 0.01 * ((m.x[i] - 10.0) ** 2 + (m.y[i] - 10.0) ** 2)  # incentiva aproximacao ao alvo
        total += (sin_term + penal) * dt
    return total

modelo_mod.energy = Objective(rule=energia_mod, sense=minimize)
resultado_mod = solver.solve(modelo_mod)

energia_modificada = value(modelo_mod.energy)
x_mod = [value(modelo_mod.x[t]) for t in modelo_mod.t]
y_mod = [value(modelo_mod.y[t]) for t in modelo_mod.t]

print(f"Energia com penalizacao adicional: {energia_modificada:.6f}")
print(f"Condicao do solver: {resultado_mod.solver.termination_condition}")


In [ ]:
# Visualiza o efeito da penalizacao adicional
plt.figure(figsize=(6, 6))
plt.plot(x_vals, y_vals, marker='o', label='Objetivo original')
plt.plot(x_mod, y_mod, marker='^', linestyle='--', label='Objetivo com penalizacao')
plt.title('Comparacao de Funcoes Objetivo')
plt.xlabel('x')
plt.ylabel('y')
plt.axis('equal')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


### Biblioteca de pistas personalizadasDefinimos tres cenarios para o IPOPT testar: uma pista livre, uma com checkpoint obrigatorio e outra com corredor estreito reforcando a aproximacao ao destino.

In [ ]:
# Construtor generico para modelos baseados em pistas
from pyomo.environ import Constraint
from collections import OrderedDict

# Tabela de pistas configuraveis: cada item altera restricoes e pesos da funcao objetivo
pistas = OrderedDict([
    ('Linha direta', {
        'descricao': 'Trajeto livre com restricoes de partida e chegada.',
        'sin_scale': 1.0,
        'penal': 0.0,
    }),
    ('Checkpoint central', {
        'descricao': 'Obrigatorio passar por (5,6) no meio do percurso.',
        'checkpoint': (5.0, 6.0),
    }),
    ('Corredor guiado', {
        'descricao': 'Limita o desvio lateral e inclui leve penalizacao rumo ao alvo.',
        'corredor': (-1.5, 1.5),
        'penal': 0.02,
        'sin_scale': 1.5,
    }),
])

def construir_modelo_pista(config):
    modelo = ConcreteModel()
    modelo.t = RangeSet(0, N)
    modelo.x = Var(modelo.t, domain=Reals)
    modelo.y = Var(modelo.t, domain=Reals)

    modelo.x[0].fix(0.0)
    modelo.y[0].fix(0.0)
    modelo.x[N].fix(10.0)
    modelo.y[N].fix(10.0)

    if config.get('checkpoint'):
        chk_x, chk_y = config['checkpoint']
        pivot = N // 2
        modelo.x[pivot].fix(chk_x)
        modelo.y[pivot].fix(chk_y)

    if config.get('corredor'):
        inferior, superior = config['corredor']
        modelo.corridor_sup = Constraint(modelo.t, rule=lambda m, t, up=superior: m.y[t] - m.x[t] <= up)
        modelo.corridor_inf = Constraint(modelo.t, rule=lambda m, t, low=inferior: m.y[t] - m.x[t] >= low)

    sin_scale = config.get('sin_scale', 1.0)
    penal = config.get('penal', 0.0)
    alvo_x, alvo_y = config.get('alvo', (10.0, 10.0))

    def vel2(m, i):
        dx = (m.x[i + 1] - m.x[i]) / dt
        dy = (m.y[i + 1] - m.y[i]) / dt
        return dx**2 + dy**2

    def energia(m):
        total = 0.0
        for i in range(N):
            v_i = vel2(m, i)
            v_next = vel2(m, i + 1) if i < N - 1 else v_i
            base_term = 0.5 * ((v_i + sin_scale * np.sin(t_vals[i])) + (v_next + sin_scale * np.sin(t_vals[i + 1])))
            penal_term = penal * ((m.x[i] - alvo_x) ** 2 + (m.y[i] - alvo_y) ** 2)
            total += (base_term + penal_term) * dt
        return total

    modelo.energy = Objective(rule=energia, sense=minimize)
    return modelo


In [ ]:
# Resolve cada pista e calcula estatisticas
resultados_pistas = OrderedDict()
for nome, cfg in pistas.items():
    modelo = construir_modelo_pista(cfg)
    resultado = solver.solve(modelo)
    energia_val = value(modelo.energy)
    trajetoria_x = [value(modelo.x[t]) for t in modelo.t]
    trajetoria_y = [value(modelo.y[t]) for t in modelo.t]
    resultados_pistas[nome] = {
        'descricao': cfg['descricao'],
        'energia': energia_val,
        'status': resultado.solver.status,
        'condicao': resultado.solver.termination_condition,
        'x': trajetoria_x,
        'y': trajetoria_y,
    }

    print(f"Pista: {nome}")
    print(f"  Descricao: {cfg['descricao']}")
    print(f"  Energia: {energia_val:.6f} | Condicao: {resultado.solver.termination_condition}
")

melhor_pista, melhor_infos = min(resultados_pistas.items(), key=lambda kv: kv[1]['energia'])
print(f"Melhor pista segundo o IPOPT: {melhor_pista} (energia {melhor_infos['energia']:.6f})")


In [ ]:
# Visualiza as trajetorias em cada pista
plt.figure(figsize=(7, 7))
for nome, info in resultados_pistas.items():
    plt.plot(info['x'], info['y'], marker='o', linestyle='-', label=f"{nome} (E={info['energia']:.2f})")
plt.xlabel('x')
plt.ylabel('y')
plt.title('Comparacao de pistas otimizadas')
plt.grid(True)
plt.axis('equal')
plt.legend()
plt.tight_layout()
plt.show()


### Leitura das pistas- **Linha direta:** referencia base com menor complexidade de restricoes.- **Checkpoint central:** obriga o robo a visitar `(5, 6)`, util para missoes de inspecao.- **Corredor guiado:** limita o desvio da diagonal e adiciona penalizacao leve rumo ao destino.

## Painel de insights personalizados
- Refinar `N` mostra que a trajetoria continua quase linear, com variacoes pequenas de energia atribuiveis ao termo senoidal.
- Fixar um checkpoint desloca a rota e eleva o custo, representando o impacto de missoes com paradas obrigatorias.
- O corredor guiado evidencia o aumento de energia quando restringimos a liberdade lateral, util para simular ambientes confinados.
- A penalizacao adicional e a biblioteca de pistas criam um menu de configuracoes para justificar a escolha mais eficiente em relatorios tecnicos.
- A animacao final facilita demonstrar visualmente o movimento otimizado ao professor ou avaliador.

### Animacao da melhor pistaGeramos uma animacao simples mostrando o robo percorrendo a trajetoria selecionada pelo menor custo de energia.

In [ ]:
# Construi animacao para a pista com menor energia
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

if 'resultados_pistas' in globals() and resultados_pistas:
    melhor_nome, melhor_info = min(resultados_pistas.items(), key=lambda kv: kv[1]['energia'])
elif 'resultados_n' in globals() and resultados_n:
    melhor_nome, melhor_info = min(resultados_n.items(), key=lambda kv: kv[1]['energia'])
else:
    raise RuntimeError('Execute os cenarios antes de gerar a animacao.')

traj_x = melhor_info['x']
traj_y = melhor_info['y']
fig, ax = plt.subplots(figsize=(6, 6))
ax.set_xlim(min(traj_x) - 1, max(traj_x) + 1)
ax.set_ylim(min(traj_y) - 1, max(traj_y) + 1)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title(f'Trajetoria animada - {melhor_nome}')
ax.grid(True)
ax.set_aspect('equal')

linha, = ax.plot([], [], 'o-', color='tab:blue')
ponto, = ax.plot([], [], 'o', color='red')

def init():
    linha.set_data([], [])
    ponto.set_data([], [])
    return linha, ponto

def atualizar(frame):
    linha.set_data(traj_x[: frame + 1], traj_y[: frame + 1])
    ponto.set_data([traj_x[frame]], [traj_y[frame]])  # envolve em listas para satisfazer o matplotlib
    return linha, ponto

ani = FuncAnimation(fig, atualizar, frames=len(traj_x), init_func=init, interval=600, blit=True, repeat=False)
plt.close(fig)
HTML(ani.to_jshtml())
